# AI Algorithms Project : Optimize biscuit manufacturing

A biscuit manufacturing factory is planning to produce a series of biscuits for Christmas. Using
the same roll of dough, the factory aims to create various biscuits in different sizes and shapes.
The goal is to maximize biscuit production from a single dough roll while ensuring the highest
possible profit.
To achieve this goal, we have the following information :
— The roll of dough is of a predefined rectangular length, referred to as ’LENGTH’, representing a one-dimensional problem.
— The roll may contain irregularities, referred to as defects. Each defect has
— a position ‘x‘
— and a class, which could be one of several types (e.g., ’a’, ’b’, ’c’, ...).
— The factory aims to produce a set of biscuits. Each Biscuit can be produced an infinite
number of times, and has :
— a specific size (along the same dimension as the roll)
— a value (price)
— and a threshold for the maximum number of defects of each class it can contain (otherwise it cannot be marketed).
A solution is an assignment of biscuits on the roll. For an assignment to be valid, it needs to :
— be at integer positions
— Ensure no overlap in biscuit placement. For example, if you place a biscuit B1 of size 3 at
position ‘x=2‘, you cannot have any biscuit assigned at positions ‘x=3‘ or ‘x=4‘.
— Ensure that each biscuit placed on the roll contains fewer defects (or an equal number) of
each class than its thresholds permit. For example, a biscuit B1 of size 3 placed at ‘x=2‘
includes all defects in the interval [2, 4]. If there are 3 defects of class ’a’ in this interval,
and B1’s threshold for class ’a’ is a maximum of 2 defects, then the assignment is invalid.
— Ensure that the sum of the sizes of the assigned biscuits does not exceed the length of the
roll of dough.
The value of a solution is the sum of the values of the individual biscuits placed on it. Any part
of the roll of dough without a biscuit assigned is considered to have a value of 0.
2.1 Benchmark
For this project, the following assumptions are made :
— The length of the roll of dough is set to 500 units.
— The roll has three classes of defects (’a’, ’b’, and ’c’). The set of defects and their positions
on the roll are available in the ’defects.csv’ file.
— The biscuit manufacturing factory aims to produce 4 types of biscuits, which are :
— Biscuit 0 with a length of 4, a value 6, and maximum allowed defects as {'a' : 4, 'b' : 2, 'c' : 3}
— Biscuit 1 with a length of 8, a value 12, and maximum allowed defects as{'a' : 5, 'b' : 4, 'c' : 4}
— Biscuit 2 with a length of 2, a value 1, and maximum allowed defects as {'a' : 1, 'b' : 2, 'c' : 1}
— Biscuit 3 with a length of 5, a value 8, and maximum allowed defects as {'a' : 2, 'b' : 3, 'c' : 2}

# 1- Define the problem
Our goal here is to maximize the profit from a roll of doughby creating several biscuits while adressing specific constraints.
Our roll has a fixed length, it contains defects,
a biscuit is defined by its:

1.   size
2.   value
2.   tolerance for certain defect classes




We want to arrange the biscuits on the roll of dough optimally so as to maximize the profit.



# 2 - Python Implementation


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

Classe Biscuit : Pour représenter chaque type de biscuit avec ses propriétés (taille, valeur, seuils de défauts).
Classe Defect : Pour représenter les défauts dans le rouleau de pâte, avec des attributs pour leur position et leur classe.
Classe Roll : Pour représenter le rouleau de pâte, avec des attributs pour sa longueur, les défauts qu'il contient, et les biscuits qui lui sont assignés.
Fonction de Vérification des Contraintes : Pour s'assurer qu'une affectation de biscuits respecte toutes les contraintes (pas de chevauchement, respect des seuils de défauts, etc.).
Fonction de Calcul de la Valeur : Pour calculer la valeur totale d'une affectation de biscuits sur le rouleau.
Fonction de Placement de Biscuits : Pour essayer différentes configurations de placement de biscuits et choisir la meilleure selon la méthode choisie (heuristique ou CSP).
Fonction Principale / Algorithme de Résolution : Pour orchestrer le processus de résolution, en utilisant soit les méthodes heuristiques, soit les techniques CSP.

In [3]:
class Biscuit:
    """
    A class to represent a biscuit.
    
    Attributes
    ----------
    size : int
        the size of the biscuit
    value : int
        the value of the biscuit
    defects : dict
        the maximum number of defects of each class the biscuit can contain
    """
    
    def __init__(self, size, value, defects):
        self.size = size
        self.value = value
        self.defects = defects

    def __str__(self):
        return f'Biscuit(size={self.size}, value={self.value}, defects={self.defects})'

class Defect:
    """
    A class to represent a defect.
    
    Attributes
    ----------
    position : int
        the position of the defect
    classe : str
        the class of the defect
    """
    
    def __init__(self, position, classe):
        self.position = position
        self.classe = classe

    def __str__(self):
        return f'Defect(position={self.position}, classe={self.classe})'
        

Let's define our 4 type of biscuits that the factory is producing. categorised as biscuit 0, biscuit 1, biscuit 2 and biscuit 3.

We will also define the defects in the roll of dough as a list of Defect objects.

In [7]:
# Read the defects from the csv file

defects = pd.read_csv('defects.csv', header=None, names=['position', 'classe'])
defects = [Defect(position, classe) for position, classe in zip(defects['position'], defects['classe'])]

biscuits = [
    Biscuit(size=4, value=6, defects={'a': 4, 'b': 2, 'c': 3}),
    Biscuit(8, 12, {'a': 5, 'b': 4, 'c': 4}),
    Biscuit(2, 1, {'a': 1, 'b': 2, 'c': 1}),
    Biscuit(5, 8, {'a': 2, 'b': 3, 'c': 2})
]

In [ ]:
class Roll:
    """
    A class to represent a roll of dough.
    It contains also the information of the assigned biscuits and their positions.
    
    Attributes
    ----------
    length : int
        the length of the roll
    defects : list
        the list of defects in the roll
    assigned_biscuits : tuple(Biscuit, int)
        the list of assigned biscuits with the type of biscuit and its starting position
    
    Methods
    -------
    
    """
    
    def __init__(self, length=500, defects = defects):
        self.length = length
        self.defects = defects
        self.assigned_biscuits = []
        
    def __str__(self):
        return f'Roll(length={self.length}, defects={self.defects}, assigned_biscuits={self.assigned_biscuits})'
    
    def check_constraints(self, biscuit, position):
        """ 
        Check if an assignment of a biscuit on the roll is valid.
        It checks if the assignment respects the following constraints:
        - There is no overlap in biscuit placement
        - Each biscuit placed on the roll contains fewer defects (or an equal number) of each class than its thresholds permit
        - The sum of the sizes of the assigned biscuits does not exceed the length of the roll of dough
        
        Args:
            biscuit (Biscuit): the biscuit to be placed
            position (int): the position of the biscuit in the roll
            
        Returns:
            bool: True if the assignment is valid, False otherwise
        """
        
        # Check if there is no overlap in biscuit placement
        for assigned_biscuit in self.assigned_biscuits:
            if position <= assigned_biscuit[1] < position + biscuit.size: # if the starting position of the already placed biscuit is in the interval of the new biscuit
                return False
            if assigned_biscuit[1] <= position < assigned_biscuit[1] + assigned_biscuit[0].size: # if the starting position of the new biscuit is in the interval of the already placed biscuit
                return False
        
        # Check if the biscuit contains fewer defects (or an equal number) of each class than its thresholds permit
        for classe, threshold in biscuit.defects.items():
            for defect in self.defects:
                if defect.position in range(position, position + biscuit.size) and defect.classe == classe: # if the defect is in the interval of the biscuit, decrease the threshold of defect type
                    threshold -= 1
            if threshold < 0:
                return False
        
        
        # Check if the sum of the sizes of the assigned biscuits does not exceed the length of the roll of dough
        if sum([assigned_biscuit[0].size for assigned_biscuit in self.assigned_biscuits]) + biscuit.size > self.length:
            return False
        
        return True